In [1]:
import cv2
import itertools as it
import numpy as np
from collections import Counter
from datetime import datetime
from math import sqrt
from random import choices, randrange
from typing import Tuple
from utilities import Timer

class mx:
    class context:
        class Context:
            pass
    def cpu():
        pass
    class nd:
        class NDArray:
            pass
        def array(array, ctx):
            return array

In [2]:
max_seq_len = 19
alphabet = list(map(chr, range(ord('A'), ord('Z') + 1))) + list("'-")
alphabet_dict = {alphabet[i]: i for i in range(len(alphabet))}
alphabet_with_spaces = alphabet[:-2] + len(alphabet) // 5 * [' ']
font_face = cv2.FONT_HERSHEY_PLAIN
thickness = 1
width, height = 132, 16

def get_text_size(text: str, font_scale: float) -> Tuple[int, int]:
    (text_width, text_height), _ = cv2.getTextSize(text, font_face, font_scale, thickness)
    return text_width + thickness, text_height + thickness

def put_text(image: np.ndarray, text: str, font_scale: float) -> None:
    text_width, text_height = get_text_size(text, font_scale)
    cv2.putText(image, text, ((width - text_width) // 2, 5 + text_height), font_face, font_scale, (255,), thickness, cv2.LINE_AA)

widths = []
def make_data(n: int, ctx: mx.context.Context) -> Tuple[mx.nd.NDArray, mx.nd.NDArray]:
    def fn(_):
        while True:
            letters = []
            while len([c for c in letters if c != ' ']) < 2:
                letters = choices(alphabet_with_spaces, k=int(sqrt(randrange(2*2, 20*20))))
            if 5 < len(letters) < 19 and not randrange(5):
                letters.insert(randrange(1, len(letters) - 1), "'" if randrange(2) else '-')
            s = ''.join(letters).strip()
            font_scale = 0.6
            text_width, text_height = get_text_size(s, font_scale)
            widths.append(text_width)
            if text_width >= 104:
                font_scale = 0.48
                text_width, text_height = get_text_size(s, font_scale)
            if text_width < 104:
                break
        image = np.zeros((height, width), dtype=np.uint8)
        put_text(image, s, font_scale)
        g = (alphabet_dict[c] for c in letters if c != ' ')
        g = it.chain(g, it.repeat(-1, max_seq_len))
        return image[np.newaxis, :, :] / 255.0, list(g)[:max_seq_len]
    x, y = (mx.nd.array(np.stack(v, axis=0), ctx=ctx) for v in zip(*map(fn, range(n))))
    return x, y

with Timer('make_data'):
    inputs, outputs = make_data(999, mx.cpu())
print(inputs.min(), inputs.max(), inputs.dtype, inputs.shape, outputs.min(), outputs.max(), outputs.dtype, outputs.shape)
image = np.vstack(inputs[:22])
image = image.reshape(-1, image.shape[-1])
print(image.shape)
cv2.imshow('tesst', image)
cv2.waitKey()
cv2.destroyWindow('tesst')
np.stack(inputs, axis=0).shape, sorted(Counter(widths).items())

make_data time: 0.044001 seconds
0.0 1.0 float64 (999, 1, 16, 132) -1 27 int32 (999, 19)
(352, 132)


((999, 1, 16, 132),
 [(10, 1),
  (13, 2),
  (14, 9),
  (15, 4),
  (16, 1),
  (17, 1),
  (18, 2),
  (19, 3),
  (20, 7),
  (21, 5),
  (22, 2),
  (23, 4),
  (24, 3),
  (25, 10),
  (26, 8),
  (27, 6),
  (28, 3),
  (30, 2),
  (31, 7),
  (32, 5),
  (33, 5),
  (34, 5),
  (35, 1),
  (36, 2),
  (37, 5),
  (38, 5),
  (39, 8),
  (40, 13),
  (41, 6),
  (42, 15),
  (43, 11),
  (44, 3),
  (45, 8),
  (46, 10),
  (47, 7),
  (48, 10),
  (49, 9),
  (50, 8),
  (51, 3),
  (52, 8),
  (53, 6),
  (54, 6),
  (55, 10),
  (56, 5),
  (57, 18),
  (58, 9),
  (59, 6),
  (60, 11),
  (61, 14),
  (62, 3),
  (63, 6),
  (64, 13),
  (65, 7),
  (66, 19),
  (67, 12),
  (68, 5),
  (69, 10),
  (70, 8),
  (71, 7),
  (72, 12),
  (73, 4),
  (74, 13),
  (75, 16),
  (76, 10),
  (77, 6),
  (78, 21),
  (79, 11),
  (80, 8),
  (81, 17),
  (82, 14),
  (83, 10),
  (84, 13),
  (85, 21),
  (86, 13),
  (87, 17),
  (88, 10),
  (89, 6),
  (90, 12),
  (91, 18),
  (92, 9),
  (93, 11),
  (94, 20),
  (95, 6),
  (96, 22),
  (97, 21),
  (98, 9),
